## $\color{Green}{\text{GMD Table 3: GMSL from Modules}}$

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import netCDF4 as nc
from pandas.plotting import table 
import xarray as xr

#### $\color{blue}{\text{List of Modules and Data to be used}}$

In [2]:
# Load FACTS workflow data as a dataframe.
df = pd.read_fwf('001_IP_GMD_Table_names_Modules_Data/ModuleTest_DataFile_List_coupling.txt',comment = '#') 
#
# Label first column with numbers. (dont to preserve the table structure)
df['Num'] = np.arange(df.shape[0])
# shift column No to first position
first_col = df.pop('Num')
df.insert(0, 'Num', first_col)
df

,Num,Component,Module,sub_Module,DataFile
0,0,GSAT,fair,temperature,temperature.fair.temperature_gsat.nc
1,1,GSAT,fair,temperature,temperature.fair.temperature_oceantemp.nc
2,2,TE,tlm,sterodynamics,ocean.tlm.sterodynamics_globalsl.nc
3,3,Glaciers,ipccar5,glaciers,ar5glaciers.ipccar5.glaciers_globalsl.nc
4,4,Glaciers,emulandice,glaciers,emuglaciers.emulandice.glaciers_globalsl.nc
5,5,GrIS,bamber19,icesheets,bamber19.bamber19.icesheets_GIS_globalsl.nc
6,6,GrIS,FittedISMIP,GrIS,GrIS1f.FittedISMIP.GrIS_GIS_globalsl.nc
7,7,GrIS,emulandice,GrIS,emuGrIS.emulandice.GrIS_globalsl.nc
8,8,AIS,ipccar5,icesheets,ar5AIS.ipccar5.icesheets_AIS_globalsl.nc
9,9,AIS,bamber19,icesheets,bamber19.bamber19.icesheets_AIS_globalsl.nc


#### $\color{Red}{\text{Load Module data from ==> facts-experiments}}$

In [4]:
EXPDATE     = 221217
MODNO       = df.index.values;   # Number of datasets Module outputs
SCENARIO    = ['ssp126', 'ssp245', 'ssp585'];
#SCENARIO    = ['ssp119','ssp126', 'ssp245', 'ssp370', 'ssp585'];
YEAR        = 2100; # Percentile Year for the table.    
#
# Temp DF to store all .nc O/P.      
df_TEMP      = []
#       
for val in MODNO:
    #
    for scenario in SCENARIO:
        #
        MODULE = df["Module"][val];  SUB_MODULE = df["sub_Module"][val]; COMP = df["Component"][val]; DATAFILE = df["DataFile"][val];
        NUM = df["Num"][val];
        # Skip if data is absent.
        if df["DataFile"][val] == 'XXX':
           continue 
        #
        # Pick the data files & Import nc file to dataframe. 
        dataFOLDER  = '/projects/kopp/facts-experiments/{arg2}/coupling.{arg1}/output/'.format(arg1=scenario,arg2=EXPDATE)
        dataFILE    = 'coupling.{arg1}.'.format(arg1=scenario) + df["DataFile"][val]
        d_nc        = xr.open_dataset(dataFOLDER + dataFILE)
        d_nc
        #
        # Percentile calculation.
        percentList = [50, 17, 83]          #[50, 5, 17, 83, 95]
        # Find year index to pick SLC value
        Yind        = np.where(d_nc["years"].values == YEAR)[0][0]
        # 
        if SUB_MODULE == 'temperature':
            if  DATAFILE == 'temperature.fair.temperature_oceantemp.nc':
                    GMSL    = (d_nc["deep_ocean_temperature"][:,Yind,0].values)/1   
            elif DATAFILE == 'temperature.fair.temperature_ohc.nc':
                    GMSL    = (d_nc["ocean_heat_content"][:,Yind,0].values)/1   
            else:
                GMSL    = (d_nc["surface_temperature"][:,Yind,0].values)/1 
        else:
            GMSL    = (d_nc["sea_level_change"][:,Yind,0].values)/100       
        # Find Percentiles.
        pcntle = np.percentile(GMSL[:], percentList );    pcntle = np.around(pcntle,2)
        #
        TEMP = [NUM,COMP,MODULE, scenario] + pcntle.tolist()
        df_TEMP.append( TEMP )
df_ptile = pd.DataFrame( df_TEMP, columns=['Num','Component','Module','SSP', ] + [ f'col_{x}' for xi, x in enumerate( percentList )] )
#
df_ptile[''] = df_ptile.apply(lambda x: f'{x.col_50:2.2f} ({x.col_17:2.2f} , {x.col_83:2.2f})', axis=1 )
df_ptile1    = pd.DataFrame( df_ptile.set_index( ['Num','Component','Module','SSP'] )[''] ).unstack().swaplevel( 0,1, axis=1 )
print('Percentile of GMSL [ median(17th,83rd) ] in cm for different modules for the year ', YEAR)
df_ptile1

Percentile of GMSL [ median(17th,83rd) ] in cm for different modules for the year  2100


,,SSP,ssp126,ssp245,ssp585
,,,,,
Num,Component,Module,,,
0,GSAT,fair,"1.63 (1.35 , 1.99)","2.61 (2.19 , 3.12)","4.66 (3.96 , 5.55)"
1,GSAT,fair,"0.76 (0.53 , 1.09)","0.91 (0.63 , 1.33)","1.24 (0.85 , 1.81)"
2,TE,tlm,"1.37 (1.12 , 1.71)","1.86 (1.54 , 2.27)","2.86 (2.40 , 3.47)"
3,Glaciers,ipccar5,"0.92 (0.66 , 1.24)","1.15 (0.83 , 1.56)","1.60 (1.13 , 2.18)"
4,Glaciers,emulandice,"0.90 (0.69 , 1.09)","1.20 (1.00 , 1.42)","1.76 (1.51 , 2.01)"
5,GrIS,bamber19,"1.27 (0.70 , 3.04)","1.27 (0.70 , 3.04)","2.22 (1.02 , 5.89)"
6,GrIS,FittedISMIP,"0.75 (0.58 , 0.97)","1.00 (0.79 , 1.24)","1.39 (1.12 , 1.76)"
7,GrIS,emulandice,"0.53 (0.11 , 1.04)","0.77 (0.35 , 1.27)","1.24 (0.82 , 1.76)"
